In this notebook, we will calculate features using Trotterization with a noisy classical simulator (AerSimulator)

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# Add the fourier_learning_ibm package to the path
# This is necessary only when running the notebook in the docker container
import sys, pprint

sys.path.append("/home/jovyan/fourier_learning_ibm/")
pprint.pprint(sys.path)

# This is necessary only when running the notebook in the docker container
%cd fourier_learning_ibm/

['/home/jovyan',
 '/opt/conda/lib/python311.zip',
 '/opt/conda/lib/python3.11',
 '/opt/conda/lib/python3.11/lib-dynload',
 '',
 '/opt/conda/lib/python3.11/site-packages',
 '/home/jovyan/fourier_learning_ibm/']
/home/jovyan/fourier_learning_ibm


In [3]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
import os
from heisenberg import (
    HeisenbergModel,
    get_graph,
    get_positions,
    # get_initial_layout,
    # get_prob0,
)
from setup import setup_backend
from helper_features import run_job, get_features
from qiskit import transpile
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler, Batch
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
import pickle
import networkx as nx
from datetime import datetime, timezone
import json
import math

In [4]:
import warnings

warnings.filterwarnings("ignore")

### Config backend 

In [5]:
backend_qpu, backend_sim_noiseless, backend_sim_noisy = setup_backend(
    qpu_name="ibm_marrakesh"
)

Using backend QPU: <IBMBackend('ibm_marrakesh')>
Using backend noiseless simulator: AerSimulator('aer_simulator')

NoiseModel:
  Basis gates: ['cz', 'delay', 'for_loop', 'id', 'if_else', 'measure', 'reset', 'rz', 'switch_case', 'sx', 'x']
  Instructions with noise: ['measure', 'id', 'cz', 'sx', 'reset', 'x']
  Qubits with noise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155

### Prepare parameters and load dataset

In [6]:
n_qubits = 4
C = 3 * (n_qubits - 1)
if n_qubits == 4:
    threshold = 0.8

# Load dataset (orient="records")
df_n_step = pd.read_json(
    f"results/get_param/{n_qubits}Q/threshold{threshold:.2f}/optimal_n_step.json",
    orient="records",
)
n_samples = 55

# Note that K = n_features-1.
# n_features counts only the number of real features, also the one of t=0, i.e., constant term.
# The number of all features, including imaginary numbers, is 2K+1.
n_features = len(df_n_step)

times = df_n_step["t"].values

display(df_n_step)
print(f"n_qubits: {n_qubits}")
print(f"C: {C}")
print(f"threshold: {threshold}")
print(f"n_samples: {n_samples}")
print(f"n_features: {n_features}")
print(f"times: {times}")

# Load dataset (orient="records")
df_dataset = pd.read_json(
    f"results/get_dataset/{n_qubits}Q/dataset.json", orient="records"
)
all_Js = df_dataset["Js"].values
all_expected_values = df_dataset["expected_value"].values
n_samples = len(all_Js)
display(df_dataset)

,t,n_step,sta_fidelity
0,0.000000,1,1.000000
1,0.349066,1,0.973699
2,0.698132,2,0.949846
3,1.047198,2,0.945106
4,1.396263,3,0.921213
5,1.745329,3,0.909344
6,2.094395,4,0.889045
7,2.443461,5,0.872148
8,2.792527,5,0.856062
9,3.141593,6,0.835667


n_qubits: 4
C: 9
threshold: 0.8
n_samples: 55
n_features: 11
times: [0.         0.34906585 0.6981317  1.04719755 1.3962634  1.74532925
 2.0943951  2.44346095 2.7925268  3.14159265 3.4906585 ]


,sample,Js,expected_value,expected_value_diag,diff
0,0,"[0.5479120971, -0.12224312050000001, 0.7171958...",13.899247,13.899220,2.726160e-05
1,1,"[0.3947360581, -0.8116453042, 0.9512447033]",28.586491,28.585213,1.278298e-03
2,2,"[0.5222794040000001, 0.5721286106, -0.7437727346]",2.653078,2.653082,3.712600e-06
3,3,"[-0.0992281242, -0.2584039515, 0.8535299777]",7.222994,7.222891,1.023508e-04
4,4,"[0.2877302402, 0.6455232265, -0.11317160230000...",0.865096,0.865078,1.800550e-05
5,5,"[-0.5455225564, 0.109169574, -0.8723654878]",1.138947,1.138950,2.786300e-06
6,6,"[0.655262344, 0.2633287982, 0.5161754802]",9.060702,9.060659,4.298700e-05
7,7,"[-0.29094806370000004, 0.9413960488, 0.7862422...",4.366950,4.367027,7.663200e-05
8,8,"[0.5567669941, -0.6107225843, -0.0665579925]",4.566096,4.566077,1.851980e-05
9,9,"[-0.9123924684, -0.6914210159, 0.3660979065]",3.720388,3.720380,8.204800e-06


# Calculate Fourier feature

## Trotter simulation (Simulator, Noise free)

In [7]:
config = {
    "n_qubits": n_qubits,
    "n_samples": n_samples,
    "n_features": n_features,
    "times": times,
    "all_Js": all_Js,
    "n_step_array": df_n_step["n_step"].values,
    "backend": backend_sim_noisy,  # This is important!!
    "all_expected_values": all_expected_values,
}

In [8]:
# For noisy simulator, we need job instances instead of job ids
_, jobs, _ = run_job(config)

Preparing circuits for sample 0/55
Preparing circuits for sample 1/55
Preparing circuits for sample 2/55
Preparing circuits for sample 3/55
Preparing circuits for sample 4/55
Preparing circuits for sample 5/55
Preparing circuits for sample 6/55
Preparing circuits for sample 7/55
Preparing circuits for sample 8/55
Preparing circuits for sample 9/55
Preparing circuits for sample 10/55
Preparing circuits for sample 11/55
Preparing circuits for sample 12/55
Preparing circuits for sample 13/55
Preparing circuits for sample 14/55
Preparing circuits for sample 15/55
Preparing circuits for sample 16/55
Preparing circuits for sample 17/55
Preparing circuits for sample 18/55
Preparing circuits for sample 19/55
Preparing circuits for sample 20/55
Preparing circuits for sample 21/55
Preparing circuits for sample 22/55
Preparing circuits for sample 23/55
Preparing circuits for sample 24/55
Preparing circuits for sample 25/55
Preparing circuits for sample 26/55
Preparing circuits for sample 27/55
Pr

In [9]:
features_df = get_features(config, jobs)
display(features_df)

Post-processing sample 0/55
Post-processing sample 1/55
Post-processing sample 2/55
Post-processing sample 3/55
Post-processing sample 4/55
Post-processing sample 5/55
Post-processing sample 6/55
Post-processing sample 7/55
Post-processing sample 8/55
Post-processing sample 9/55
Post-processing sample 10/55
Post-processing sample 11/55
Post-processing sample 12/55
Post-processing sample 13/55
Post-processing sample 14/55
Post-processing sample 15/55
Post-processing sample 16/55
Post-processing sample 17/55
Post-processing sample 18/55
Post-processing sample 19/55
Post-processing sample 20/55
Post-processing sample 21/55
Post-processing sample 22/55
Post-processing sample 23/55
Post-processing sample 24/55
Post-processing sample 25/55
Post-processing sample 26/55
Post-processing sample 27/55
Post-processing sample 28/55
Post-processing sample 29/55
Post-processing sample 30/55
Post-processing sample 31/55
Post-processing sample 32/55
Post-processing sample 33/55
Post-processing sample 3

,sample_id,f_0 Re,f_1 Re,f_1 Im,f_2 Re,f_2 Im,f_3 Re,f_3 Im,f_4 Re,f_4 Im,...,f_6 Im,f_7 Re,f_7 Im,f_8 Re,f_8 Im,f_9 Re,f_9 Im,f_10 Re,f_10 Im,expected_value
0,0,0.978516,0.664279,0.359029,0.189744,0.318757,-0.056674,0.013426,-0.088716,-0.031977,...,0.181723,-0.682220,-0.052918,-0.446197,-0.319944,-0.016217,-0.202470,0.152605,0.128155,13.899247
1,1,0.976562,0.514052,0.512191,-0.146759,0.273465,-0.263167,-0.140530,0.132419,-0.256705,...,-0.006527,0.147678,-0.063113,0.068220,-0.032827,-0.174492,0.193740,-0.420429,0.168291,28.586491
2,2,0.978516,0.727397,-0.197708,0.263911,-0.309515,-0.056023,-0.273511,-0.066432,-0.236813,...,-0.347491,0.133656,-0.289935,-0.136388,-0.182237,-0.457015,0.033255,-0.496077,0.183089,2.653078
3,3,0.988281,0.756097,0.270295,0.227613,0.225188,-0.215981,-0.068888,-0.329358,-0.476657,...,-0.600875,0.339538,-0.229288,0.170883,0.137486,-0.085485,0.267475,-0.272947,0.165783,7.222994
4,4,0.981445,0.917174,-0.148827,0.827621,-0.249945,0.674519,-0.375019,0.496129,-0.493445,...,-0.545192,0.179152,-0.471213,0.146808,-0.377828,0.090882,-0.280034,0.058481,-0.193527,0.865096
5,5,0.985352,0.653368,-0.383984,0.066483,-0.223974,-0.056923,0.117185,-0.036344,0.050033,...,-0.002415,-0.257092,0.153768,0.139211,-0.074013,0.191531,-0.522009,-0.125028,-0.582813,1.138947
6,6,0.974609,0.741068,0.231158,0.365310,0.237188,0.188586,0.033459,0.146027,-0.016863,...,0.443474,-0.456069,0.418414,-0.525425,0.169392,-0.339246,0.058155,-0.098405,0.125965,9.060702
7,7,0.978516,0.795815,-0.151199,0.427892,-0.274946,0.154586,-0.308797,0.157545,-0.224021,...,0.260986,0.358661,0.302128,0.248739,0.170083,0.340470,-0.092760,0.373054,-0.125672,4.366950
8,8,0.978516,0.823524,0.335711,0.471174,0.502309,0.072539,0.467930,-0.202837,0.278558,...,-0.292935,0.229921,-0.404795,0.524696,-0.352360,0.738839,-0.166428,0.709807,0.099306,4.566096
9,9,0.971680,0.727097,0.055665,0.279356,0.146710,-0.005708,0.278592,0.046458,0.255557,...,-0.202130,0.323213,-0.227863,0.203663,-0.111365,0.198268,0.123644,0.208900,0.029891,3.720388
